# Download the required libraries

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


# Create simple custom dataset

In [0]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

# Create label set for our data
You can see that the first 8 items in the sentiment array contain 1, which correspond to positive sentiment. The last 8 items are zero that correspond to negative sentiment.

In [0]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])


# Find the total number of words in our corpus 
The first parameter to the Embedding() layer is the vocabulary, or number of unique words in the corpus. 

In [5]:
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

all_words = []
for sent in corpus:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Set all_words to unique_words
unique_words = 45 I'll add a buffer of 5 to our vocabulary size and will set the value of vocab_length = 50.

In [8]:
unique_words = set(all_words)
vocab_length = 50
print(len(all_words))
print(len(unique_words))

71
45


In [9]:
embedded_sentences = [one_hot(sent, vocab_length) for sent in corpus]
print(embedded_sentences )

[[34, 37, 7, 37, 31], [41, 7, 31, 2, 12, 16, 30], [3, 44, 24, 30, 37, 36], [34, 36], [30, 4, 27, 10, 12, 16, 30], [42, 4, 2, 44], [48, 24, 46, 4, 19, 31], [30, 37, 4, 4, 31], [20, 31], [39, 27, 4], [5, 16], [30, 31, 29, 33], [12, 6, 17, 16, 41, 31], [41, 31, 31, 20], [12, 47, 17, 38], [41, 31, 37, 5]]


# Find the largest sentence
The embedding layer expects sentences to be of equal size. However, our encoded sentences are of different sizes. One way to make all the sentences of uniform size is to increase the lenght of all the sentences and make it equal to the length of the largest sentence. Let's first find the largest sentence in our corpus and then we will increase the length of all the sentences to the length of the largest sentence.

In [0]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

# Make all the sentences of equal size
we will add zeros to the empty indexes that will be created as a result of increasing the sentence length. To append the zeros at the end of the sentencses, we can use the pad_sequences method. The first parameter is the list of encoded sentences of unequal sizes, the second parameter is the size of the longest sentence or the padding index, while the last parameter is padding where you specify post to add padding at the end of sentences.

In [11]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[34 37  7 37 31  0  0]
 [41  7 31  2 12 16 30]
 [ 3 44 24 30 37 36  0]
 [34 36  0  0  0  0  0]
 [30  4 27 10 12 16 30]
 [42  4  2 44  0  0  0]
 [48 24 46  4 19 31  0]
 [30 37  4  4 31  0  0]
 [20 31  0  0  0  0  0]
 [39 27  4  0  0  0  0]
 [ 5 16  0  0  0  0  0]
 [30 31 29 33  0  0  0]
 [12  6 17 16 41 31  0]
 [41 31 31 20  0  0  0]
 [12 47 17 38  0  0  0]
 [41 31 37  5  0  0  0]]


# Create a simple text classification model with an embedding layer and no hidden layers.



In [0]:
model = Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# Compile the model and print the summary of our model
You can see that the first layer has **1000** trainable parameters. This is because our **vocabulary size is 50** and each word will be presented as a **20 dimensional vector**. Hence the total number of trainable parameters will be 1000. Similarly, the output from the embedding layer will be a sentence with 7 words where each word is represented by a 20 dimensional vector. However, when the **2D output is flattened, we get a 140 dimensional vector (7 x 20)**. The flattened vector is directly connected to the dense layer that contains 1 neuran.

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten_1 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


#Train the model on our data using the fit method

In [17]:
model.fit(padded_sentences, sentiments, epochs=100, verbose=1)


Epoch 1/100
16/16 [==============================] - 0s 20ms/step - loss: 0.7023 - acc: 0.3750
Epoch 2/100
16/16 [==============================] - 0s 289us/step - loss: 0.6985 - acc: 0.3750
Epoch 3/100
16/16 [==============================] - 0s 190us/step - loss: 0.6947 - acc: 0.4375
Epoch 4/100
16/16 [==============================] - 0s 107us/step - loss: 0.6909 - acc: 0.5000
Epoch 5/100
16/16 [==============================] - 0s 110us/step - loss: 0.6871 - acc: 0.5625
Epoch 6/100
16/16 [==============================] - 0s 157us/step - loss: 0.6834 - acc: 0.6875
Epoch 7/100
16/16 [==============================] - 0s 79us/step - loss: 0.6798 - acc: 0.8125
Epoch 8/100
16/16 [==============================] - 0s 84us/step - loss: 0.6760 - acc: 0.8125
Epoch 9/100
16/16 [==============================] - 0s 81us/step - loss: 0.6723 - acc: 0.8750
Epoch 10/100
16/16 [==============================] - 0s 78us/step - loss: 0.6686 - acc: 0.9375
Epoch 11/100
16/16 [========================